In [0]:
%pip install -U -qqq langchain_core langchain_databricks langchain_community
%restart_python

In [0]:
%sql
SELECT * FROM `Dais`.bright_initiative.airbnb_properties_information
WHERE
  location = 'Chicago, Illinois, United States'
  AND host_number_of_reviews > 1000
  AND EXISTS(reviews, review -> review ILIKE '%wheelchair%')
LIMIT 5;

In [0]:
import pandas as pd
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_databricks import ChatDatabricks
from databricks.sdk import WorkspaceClient
import os

# configure workspace tokens
w = WorkspaceClient()
os.environ["DATABRICKS_HOST"] = w.config.host
os.environ["DATABRICKS_TOKEN"] = w.tokens.create(comment="for model serving", lifetime_seconds=1200).token_value

llm = ChatDatabricks(endpoint="databricks-llama-4-maverick")

def format_context(df: pd.DataFrame) -> str:
    """
    Converts the DataFrame into a JSON string to ensure all data is passed
    to the model without truncation. JSON is also a great format for structured data
    like you have in 'description_by_sections'.
    """
    return df.to_json(orient='records', indent=2)

def find_accessible_airbnb_properties(location: str) -> pd.DataFrame:
  """
  Queries the Bright Initiative Airbnb dataset for properties in a specific location
  that have reviews mentioning "wheelchair".
  """
  query = f"""
    SELECT
      listing_name,
      location_details,
      location,
      details,
      description_by_sections,
      reviews
    FROM `Dais`.bright_initiative.airbnb_properties_information
    WHERE
      location ILIKE '%{location}%'
      AND host_number_of_reviews > 1000
      AND EXISTS(reviews, review -> review ILIKE '%wheelchair%')
    LIMIT 5
  """
  return format_context(spark.sql(query).toPandas())

# Define the prompt template for the LLM
prompt_template = PromptTemplate.from_template(
  """
  You are a helpful assistant for accessible travel. Your goal is to summarize potential Airbnb listings for a user.

  The following listing *mention* wheelchairs but may not actually be accessible. Closely review the descriptions and review,
  and then summarize the accessibility features (or lack thereof).

  Here is the JSON data:
  {context}
  """
)

llm = ChatDatabricks(endpoint="databricks-llama-4-maverick")

# This is our simple "agentic" chain
chain = (
    find_accessible_airbnb_properties
    | prompt_template
    | llm
    | StrOutputParser()
)

# Let's run the chain for Chicago!
result = chain.invoke("Chicago")

print(result)

In [0]:
spark.sql("""
CREATE OR REPLACE TEMP VIEW medicare_benefits_analysis AS
WITH benefit_codes AS (
  SELECT array(
    '14c10', '14c11', '14c12', '14c13', '14c14', '14c15', '14c16', '14c17', '14c18', '14c19',
    '14c20', '14c21', '14c22', '14c8', '14c9', '17a1', '17a2', '17b1', '17b2', '17b3',
    '17b4', '17b5', '18a1', '18a2', '18b1', '18b2', '18b3', '18b4', '18c',
    '13a', '7b1', '7b2', '16b1', '16b2', '16b3', '16b4', '16b5', '16b6',
    '11a1', '11a2', '14c4', '6-1', '6-2', '6-3', '6-4', '13b',
    '13i1', '13i2', '13i3', '13i4', '13i5', '13i6', '13i7', '13i8', '13i9', '13i10',
    '13i11', '13i12', '13i13', '13i14', '13i15', '10b1', '10b2'
  ) AS codes
),
latest_bids AS (
  SELECT *
  FROM (
    SELECT *,
           ROW_NUMBER() OVER (PARTITION BY bid_id ORDER BY version DESC) as rn
    FROM `dais-hackathon-2025`.mimilabs.pbp_section_d
    WHERE YEAR(mimi_src_file_date) = 2025
  )
  WHERE rn = 1
),
processed_bids AS (
  SELECT
    lb.*,
    filter(split(regexp_replace(COALESCE(lb.pbp_d_combo_nmc_cats_1, ''), '\s+', ''), ';'), x -> x != '') AS clean_cats_1,
    filter(split(regexp_replace(COALESCE(lb.pbp_d_combo_nmc_cats_2, ''), '\s+', ''), ';'), x -> x != '') AS clean_cats_2,
    filter(split(regexp_replace(COALESCE(lb.pbp_d_combo_nmc_cats_3, ''), '\s+', ''), ';'), x -> x != '') AS clean_cats_3,
    filter(split(regexp_replace(COALESCE(lb.pbp_d_combo_nmc_cats_4, ''), '\s+', ''), ';'), x -> x != '') AS clean_cats_4,
    filter(split(regexp_replace(COALESCE(lb.pbp_d_combo_nmc_cats_5, ''), '\s+', ''), ';'), x -> x != '') AS clean_cats_5
  FROM latest_bids lb
  CROSS JOIN benefit_codes bc
  WHERE arrays_overlap(bc.codes,
    filter(split(regexp_replace(CONCAT_WS(';',
          COALESCE(lb.pbp_d_combo_nmc_cats_1, ''), COALESCE(lb.pbp_d_combo_nmc_cats_2, ''),
          COALESCE(lb.pbp_d_combo_nmc_cats_3, ''), COALESCE(lb.pbp_d_combo_nmc_cats_4, ''),
          COALESCE(lb.pbp_d_combo_nmc_cats_5, '')), '\s+', ''), ';'), x -> x != '')
  )
),
final_benefits AS (
  SELECT
    pb.pbp_a_hnumber,
    pb.pbp_a_plan_identifier,
    array_distinct(flatten(array(pb.clean_cats_1, pb.clean_cats_2, pb.clean_cats_3, pb.clean_cats_4, pb.clean_cats_5))) AS all_benefits
  FROM processed_bids pb
)
SELECT
  pbp_a_hnumber,
  pbp_a_plan_identifier,
  all_benefits,
  arrays_overlap(all_benefits, array('13a')) AS has_acupuncture,
  arrays_overlap(all_benefits, array('7b1', '7b2')) AS has_chiropractic,
  arrays_overlap(all_benefits, array('16b1', '16b2', '16b3', '16b4', '16b5', '16b6')) AS has_dental,
  arrays_overlap(all_benefits, array('11a1', '11a2')) AS has_dme,
  arrays_overlap(all_benefits, array('14c4')) AS has_fitness,
  arrays_overlap(all_benefits, array('6-1', '6-2', '6-3', '6-4')) AS has_home_health,
  arrays_overlap(all_benefits, array('13b', '13i1', '13i2', '13i3', '13i4', '13i5', '13i6', '13i7', '13i8', '13i9', '13i10', '13i11', '13i12', '13i13', '13i14', '13i15')) AS has_otc_ssbci,
  arrays_overlap(all_benefits, array('10b1', '10b2')) AS has_transportation
FROM final_benefits
""")

print("Temporary view 'medicare_benefits_analysis' created successfully.")

In [0]:

%sql
-- Find the first 10 plans that offer fitness benefits
SELECT *
FROM
  medicare_benefits_analysis
WHERE
  has_fitness = true
LIMIT 10;

In [0]:


import pandas as pd
from typing import List
from langchain_community.tools import tool
from langchain_databricks import ChatDatabricks
from langchain.agents import AgentExecutor, create_react_agent
from langchain_core.prompts import PromptTemplate
from databricks.sdk import WorkspaceClient
import os
import ast

@tool
def find_plans_with_benefits(required_benefits: str) -> str:
  """
  Finds Medicare Advantage plans that include all of the specified supplemental benefits.
  Use this tool when a user asks to find plans with a specific list of benefits.
  The input must be a Python list of benefit names provided as a string, chosen from:
  'acupuncture', 'chiropractic', 'dental', 'dme', 'fitness', 'home_health', 'otc_ssbci', 'transportation'.
  For example: "['dental', 'fitness']"
  """
  try:
    # The LLM often returns a string representation of a list, e.g., "['dental', 'fitness']".
    # ast.literal_eval safely evaluates this string into an actual Python list.
    benefits_list = ast.literal_eval(required_benefits)
    if not isinstance(benefits_list, list):
        raise ValueError("Input could not be parsed into a list.")
  except (ValueError, SyntaxError) as e:
    # If parsing fails, return an error message to the agent so it can retry.
    return f"Error: Invalid input format. Expected a string representation of a list. {e}"

  # Build a WHERE clause from the list of required benefits
  where_conditions = [f"has_{benefit.strip()} = true" for benefit in benefits_list]
  where_clause = " AND ".join(where_conditions)

  query = f"""
    SELECT pbp_a_hnumber, pbp_a_plan_identifier, all_benefits
    FROM medicare_benefits_analysis
    WHERE {where_clause}
    LIMIT 10
  """
  print(f"Executing query: {query}") # For debugging
  df = spark.sql(query).toPandas()
  return df.to_json(orient='records', indent=2)

# Configure workspace client for authentication
w = WorkspaceClient()
os.environ["DATABRICKS_HOST"] = w.config.host
os.environ["DATABRICKS_TOKEN"] = w.tokens.create(comment="for model serving", lifetime_seconds=1200).token_value

# The LLM is the "brain" of the agent
llm = ChatDatabricks(endpoint="databricks-llama-4-maverick", max_tokens=2048)

# The Agent needs a list of tools it can use
tools = [find_plans_with_benefits]

# This prompt template tells the agent how to reason about using tools
# This is a standard "ReAct" (Reasoning + Acting) prompt
prompt = PromptTemplate.from_template("""
Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}
""")

# Create the agent by combining the LLM, tools, and prompt
agent = create_react_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

# Let's ask our agent a question in natural language!
response = agent_executor.invoke({
    "input": "Can you find me some plans that have dental, fitness, and transportation benefits?"
})

print("\n\n--- Final Answer ---")
print(response['output'])

In [0]:
%sql
select * from `dais-hackathon-2025`.nimble.dbx_amazon_serp_daily limit 5;

In [0]:
import pandas as pd
from langchain_community.tools import tool
from langchain_databricks import ChatDatabricks
from langchain.agents import AgentExecutor, create_react_agent
from langchain_core.prompts import PromptTemplate
from databricks.sdk import WorkspaceClient
import os


@tool
def find_products_by_keyword(search_term: str) -> str:
  """
  Searches the Nimble e-commerce dataset for products matching a keyword search_term.
  Use this to find products based on a user's request.
  The input must be a single string. For example: "clip on reading light"
  """
  query = f"""
    SELECT
      productId,
      product_name,
      price,
      rating,
      review_count,
      product_url
    FROM `dais-hackathon-2025`.nimble.dbx_amazon_serp_daily
    WHERE
      product_name ILIKE '%{search_term}%'
    LIMIT 5
  """
  print(f"Executing query: {query}") # For debugging
  df = spark.sql(query).toPandas()
  return df.to_json(orient='records', indent=2)

  w = WorkspaceClient()
os.environ["DATABRICKS_HOST"] = w.config.host
os.environ["DATABRICKS_TOKEN"] = w.tokens.create(comment="for model serving", lifetime_seconds=1200).token_value

# The LLM is the "brain" of the agent
llm = ChatDatabricks(endpoint="databricks-llama-4-maverick", max_tokens=2048)

# The Agent needs a list of tools it can use
tools = [find_products_by_keyword]

# This prompt template tells the agent how to reason and use tools
prompt = PromptTemplate.from_template("""
Answer the following questions as best you can. You have access to the following tools:

{tools}

Your process should be to first think of a few different, specific search terms based on the user's question.
Then, use the `find_products_by_keyword` tool for each of those keywords, one at a time.
After you have gathered all the product information from your searches, combine the results into a single, helpful summary for the user.

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: After you find the products from all your searches, your final answer should summarize them and suggest that a good next step would be to use a web browsing tool to analyze the product pages for more detailed information.

Begin!

Question: {input}
Thought:{agent_scratchpad}
""")

# Create the agent by combining the LLM, tools, and prompt
agent = create_react_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

# Let's ask our agent to find a product related to accessibility
response = agent_executor.invoke({
    "input": "I'm looking for a tool to help my elderly parent open tight jars."
})

print("\n\n--- Final Answer ---")
print(response['output'])

In [0]:
import os
import pandas as pd
from langchain_community.tools import tool
from langchain_databricks import ChatDatabricks
from langchain.agents import AgentExecutor, create_react_agent
from langchain_core.prompts import PromptTemplate
from databricks.sdk import WorkspaceClient

# --- set up Databricks auth ---
w = WorkspaceClient()
os.environ["DATABRICKS_HOST"] = w.config.host
os.environ["DATABRICKS_TOKEN"] = w.tokens.create(
    comment="for model serving", lifetime_seconds=1200
).token_value

@tool
def find_places_by_keyword(search_term: str) -> str:
    """
    Searches the Nimble Google Maps dataset for places matching a keyword
    across title, offerings, business descriptions, and categories.
    Input: a single string, e.g. "playground wheelchair accessible"
    Returns: up to 5 matching places as JSON.
    """
    query = f"""
    SELECT
      title,
      street_address,
      city,
      country,
      latitude,
      longitude,
      rating,
      number_of_reviews,
      price_level,
      place_information.website_url  AS website_url,
      place_information.reviews_link AS reviews_link,
      phone_number,
      place_url,
      business_category_ids,
      business_status
    FROM (
      SELECT
        *,
        transform(offerings, o -> o.display_name)   AS offering_names,
        business_description                       AS descriptions,
        business_category_ids                      AS categories
      FROM `dais-hackathon-2025`.nimble.dbx_google_maps_search_daily
    ) t
    WHERE
      exists(t.offering_names, x -> x ILIKE '%{search_term}%')
      OR exists(t.descriptions,    x -> x ILIKE '%{search_term}%')
      OR exists(t.categories,      x -> x ILIKE '%{search_term}%')
    LIMIT 5
    """
    print(f"Executing query: {query}")  # debug
    df = spark.sql(query).toPandas()
    return df.to_json(orient='records', indent=2)


# --- LLM & tools setup ---
llm = ChatDatabricks(endpoint="databricks-llama-4-maverick", max_tokens=2048)
tools = [find_places_by_keyword]

# --- updated prompt template ---
prompt = PromptTemplate.from_template("""
Answer the following question using your tools:

{tools}

Your process should be:
1. Think of a few specific search terms for the user's question.
2. Call `find_places_by_keyword` for each term individually, as in "playground", "wheelchair accessible" and relevant combinations.
3. Combine all observations into a structured recommendation.

Use this format:

Question: the input question
Thought: your reasoning
Action: the tool to call (must be one of [{tool_names}])
Action Input: the input string
Observation: the tool’s output
... (repeat Thought/Action/Action Input/Observation as needed)
Thought: I now know the final answer
Final Answer: your comprehensive recommendation and next steps

Begin!

Question: {input}
Thought:{agent_scratchpad}
""")

# --- build and run the agent ---
agent = create_react_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

# Example invocation:
response = agent_executor.invoke({
    "input": "Find a good accessible playground in San Francisco for toddlers."
})
print("\n\n--- Final Answer ---")
print(response['output'])
